In [ ]:
import os
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# GLOBALS
SUPPLEMENTARY_TABLES_PATH="/Users/ochapman/projects/pedpancan_ecdna/data/Supplementary Tables.xlsx"
AC_BED_DIR_PATH="/Users/ochapman/projects/pedpancan_ecdna/data/source/AmpliconClassifier/pedpancan_classification_bed_files"
AC_BED_DEST_PATH="/Users/ochapman/projects/pedpancan_ecdna/data/source/AmpliconClassifier/beds_by_tumor_type"

TUMOR_TYPES=["RBL"]
AMPLICON_TYPES=["ecDNA","BFB","Complex-non-cyclic","Linear","unknown"]

In [ ]:
def import_patients():
    return pd.read_excel(SUPPLEMENTARY_TABLES_PATH,sheet_name="1. Patients",index_col=0)
def import_biosamples():
    return pd.read_excel(SUPPLEMENTARY_TABLES_PATH,sheet_name="2. Biosamples",index_col=0)
def import_amplicons():
    return pd.read_excel(SUPPLEMENTARY_TABLES_PATH,sheet_name="3. Amplicons")
def import_genes():
    return pd.read_excel(SUPPLEMENTARY_TABLES_PATH,sheet_name="4. Gene amplifications")
BIOSAMPLES = import_biosamples()

In [ ]:
def makedir(path):
    try:
        os.makedirs(path, exist_ok=True)
        print(f"successfully created directory {path}")
    except OSError as error:
        print(f"directory {path} already exists")
def get_bed_files(tumor_type,amplicon_type):
    biosamples = BIOSAMPLES[(BIOSAMPLES.cancer_type == tumor_type) & (BIOSAMPLES.in_unique_tumor_set)].index
    files = []
    for filename in os.listdir(AC_BED_DIR_PATH):
        if (amplicon_type in filename) and (any(substring in filename for substring in biosamples)):
            files.append(filename)
    return files
def symlink_bed_files():
    # create root
    makedir(AC_BED_DEST_PATH)
    # iterate through tumor types, amplicon types
    for tt in TUMOR_TYPES:
        for at in AMPLICON_TYPES:
            # get set of bed files of this tumor and amplicon type
            bed_files = get_bed_files(tt,at)
            if len(bed_files) > 0:
                path=os.path.join(AC_BED_DEST_PATH,tt,at)
                makedir(path)
                for bed in bed_files:
                    src=os.path.join(AC_BED_DIR_PATH,bed)
                    dst=os.path.join(path,bed)
                    #print(f"os.symlink(src={src},dst={dst})")
                    os.symlink(src=src,dst=dst)
            


In [ ]:
symlink_bed_files()